图片分类：
    图片分类不需要定位
目标检测：
    目标检测需要定位出物体的位置，也就是相当于把物体的bbox检测出来，
    目标检测是要把所有图片中的物体都识别定位出来


迁移学习：
    就是把一个任务训练好的参数，拿到另外一个任务，作为神经网络的初始参数值,
    这样相比于你直接采用随机初始化的方法，精度可以有很大的提高


定位的精度评价公式：IOU
    IOU=(A∩B)/(A∪B)
    就是矩形框A、B的重叠面积占A、B并集的面积比例:


非极大值抑制：
    作用：判别哪些矩形框是没用的,并丢弃
    原理：
        例子：假设有6个矩形框，根据分类器类别分类概率做排序，
        从小到大分别属于车辆的概率分别为[A,B,C,D,E,F]
            1. 从F开始,判断A-E 与F 的重叠度IOU 是否大于设定的阈值
            2. 假设 B,D与F 的IOU超过阈值,那就留下F,并标记,丢弃B,D.
            3. 此时剩余[A,C,E,    已标： F].   从A,C,E 中选择概率最大的E
            判断E和A,C  假设A和E的IOU大于阈值,丢掉 A.  .
            4. 此时剩余[C ,     已标： E ,F]  留下三个矩形框
            5.重复未标部分  C无法与别人比较IOU  [  已标：C,E,F ]


物体检测,目标分类
    可以用迁移学习,或者常规的决策树.对已经标出的矩形框数据来执行分类任务

预备知识点：
    1.Ground Truth：t是标注.正确的t标注是ground truth， 错误的标记则不是
    2.IOU交并比（Intersection over Union)   IOU = 交集/并集,重合度越大，说明二者越相近
    3.

整体思路：
   目标检测需要定位出物体的位置，这种就相当于回归问题，求解一个包含物体的方框.
   解决的问题：
        定位和识别

    样本的类别是21个（包括了背景)   20(物体)+背景
RCNN(Region CNN)
    步骤简述：
        1.找出候选框,首先用select search方法在每张图像上选取约2000个region proposal
        2.region proposal进行尺寸变换到一个统一尺寸227*227
        3.然后采用CNN提取每个候选框中图片的特征向量(4096维),保留f7的输出特征2000*4096维
        4.采用svm算法对各个候选框中的物体进行分类识别
          针对每个类别（一共20类）训练一个SVM分类器，以f7层的输出作为输入，训练SVM的权重4096*20维
        5.测试的时候会对这个得分输出做NMS（non-maximun suppression），简单讲就是去掉重复框的过程,其实也可以采用

    实现方式(训练过程)：
        1.准备region proposal .对于训练集中的所有图像，采用selective search方式来获取，最后每个图像得到2000个region proposal.
        2.准备正负样本,如果某个region proposal和当前图像上的所有ground truth中重叠面积最大的那个的IOU大于等于0.5,正样本包含了ground truth
        3.预训练, 采用迁移学习,在别人训练好的参数上添加层次,RCNN 采用的是12年的AlexNet来学习特征
          包含5个卷积层和2个全连接层.
        4.fine - tuning  将2中得到的样本进行尺寸变换，使得大小一致,将region proposal变形成227*227,作为3中预训练好的网络的输入，继续训练网络(迁移学习).，迁移的时候要做修改，将最后一个全连接层的输出由1000改成21，其他结构不变。训练结束后保存f7的特征
        5.针对每个类别训练一个SVM的二分类器,输入是f7的特征，f7的输出维度是2000*4096，输出的是是否属于该类别，训练结果是得到SVM的权重矩阵W，W的维度是4096*20。这里负样本的选定和前面的有所不同，将IOU的阈值从0.5改成0.3，即IOU<0.3的是负样本，正样本是Ground Truth.
        在SVM阶段是由于SVM适用于小样本，所以设置0.3会更严格一点.

    测试方式：
        1.准备一张图片,selective search方式来获取2000个region proposal
        2.对所有region proposal变换到固定尺寸并作为已训练好的CNN网络的输入，得到f7层的4096维特征，所以f7层的输出是2000*4096
        3.采用训练好的svm对特征打分,SVM的weight matrix是4096*N，N是类别数，这里一共有20个SVM，N=20注意不是21。得分矩阵是2000*20，表示每个region proposal属于某一类的得分.
        4.采用non-maximun suppression（NMS）对得分矩阵中的每一列中的region proposal进行剔除，就是去掉重复率比较高的几个region proposal，得到该列中得分最高的几个region proposal

    问题与改进方向：
        问题
            流程较多,
            包含了region proposal的选取
            迁移学习,训练卷积神经网络
            针对识别训练了SVM和regressor
        改进方向
            region proposal之后就可以丢弃重复的框,无需浪费资源全部运算2000个卷积
            流程太多,是否可以不采用SVM完成识别分类
            是否可以建立一个由始而终的目标检测和分类的网络
            中间数据是否可以完全在内存或者运存上,不需要存储在硬盘上以加快速度

In [ ]:
https://blog.csdn.net/u014380165/article/details/72851319